In [7]:
import pandas as pd
import numpy as np
from decimal import Decimal

x_chat = chatGPT()
x_gemini = gemini()




In [61]:
df_exp = pd.read_csv('../output/onet/gpt_exposure_embeddings.csv')
# df_exp = df_exp[df_exp.Title=="Fundraising Managers"]
df_exp.Embeddings = df_exp.Embeddings.apply(lambda x: x.strip("[]").split(", "))
embeddings = []
for x in df_exp.Embeddings:
    embeddings.append([float(y) for y in x])


embeddings = np.array(embeddings)
df_exp.Embeddings = embeddings
embeddings = np.vstack(embeddings)
embeddings.shape


(11582, 3072)

In [65]:
sample_df = pd.read_csv('../output/df_with_examples.csv')
sample_df.loc[:, ["Job1_ONET", "Job2_ONET", "Job3_ONET"]] = 'N/A'

# list(sample_df.Job1_embedding.apply(ast.literal_eval))
for i in range(3):
    sample_df[f"Job{i+1}_embedding"] = sample_df[f"Job{i+1}_embedding"].apply(lambda x: [Decimal(y) for y in x.strip("[]").split(", ")])
    sample_df[f"Job{i+1}_title_embedding"] = sample_df[f"Job{i+1}_title_embedding"].apply(lambda x: [Decimal(y) for y in x.strip("[]").split(", ")])




In [66]:

results = []
for index, row in list(sample_df.iterrows())[0:]:
    print("***********************")
    print(f"Processing {index}")
    name = row["organization name"]
    print(name)
    print(row["generated_description"])
        
    job_results = []
    for i in range(3): 
        example = row[f"Example{i+1}"]
        job = row[f'Job{i+1}']
        job_title = row[f'Job{i+1}_title']
        print(f"\nProcessing Example: {example} \nJob: {job}\n")
        job_embedding = np.array(row[f'Job{i+1}_embedding']).reshape(1,-1)
        job_title_embedding = np.array(row[f'Job{i+1}_title_embedding']).reshape(1,-1)
        cosine_sim = cosine_similarity(job_embedding, embeddings, 'cosine')
        
        top3 = sorted(zip(cosine_sim[0], df_exp[['Title', 'Task']].values), reverse=True)[:3]
        for cosine_sim_job, onet in top3:
            onet_title_embedding = np.array([x_chat.get_embedding(onet[0])])
            cosine_sim_title = cosine_similarity(job_title_embedding, onet_title_embedding, 'cosine')[0][0]
            print(f"{onet[0]}: {onet[1]} \nCosine Similarity: {cosine_sim_job}, Person Cosine Similarity: {cosine_sim_title}")
            results.append([name,example,row[f"situation{i+1}_conf_interval"], row[f"situation{i+1}_conf_interval_reasoning"],job,onet[0],onet[1],job_title, cosine_sim_job,cosine_sim_title])
 

    if index % 5 == 0:
        print("**Saving Results**\n\n")
        onet_df = pd.DataFrame(results, columns=["organization_name","example","situation_conf_interval", "situation_conf_interval_reasoning","job","onet_title","onet_task","example_job_title","task_similarity", "job_title_similarity"])
        onet_df.to_csv("../output/onet_df.csv", index=False)
        
    print("***********************\n\n")


***********************
Processing 0
NonprofitsHQ
NonprofitsHQ is an all-in-one platform designed specifically for nonprofits, offering a suite of AI-powered tools to simplify and streamline operations like fundraising, donor management, program management, and communication. The platform aims to save nonprofits time and money by automating tasks and providing valuable insights.

Processing Example: A development officer uses the platform to automate the process of creating personalized thank-you letters for donors, generating reports on donor engagement, and identifying potential new donors based on past giving patterns. 
Job: Development Officers who manually create thank-you letters, generate donor reports, and identify potential donors.

Fundraisers: Write and send letters of thanks to donors. 
Cosine Similarity: 0.6439689913482416, Person Cosine Similarity: 0.9999999999999973
Fundraisers: Identify and build relationships with potential donors. 
Cosine Similarity: 0.587215717281812